In [109]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import requests
import urllib
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common import action_chains
import re
import time
from selenium.webdriver.chrome.options import Options

In [114]:
def dynamicSoup(url, link_driver, n_boucle):
    options = Options()
    # options.add_argument("headless")
    options.add_argument("--start-maximized")
    driver = webdriver.PhantomJS(link_driver)
    driver.set_window_size(1400,1000)
    #driver = webdriver.Chrome(link_driver, options=options)
    driver.get(url)
    y = 0
        
    for _ in range(n_boucle):
        # Scroll down to bottom
        driver.execute_script("window.scrollTo("+str(y)+","+str(y+2000)+")")
        time.sleep(2)
        y += 2000
    return bs(driver.page_source, 'html.parser'), driver

In [118]:
def clickShow(driver):
    driver.find_element_by_class_name("_3j0u").click()
    elements = driver.find_elements_by_class_name("_4ssp")
    action = action_chains.ActionChains(driver)
    for el in elements:
        time.sleep(2)
        try:
            action.move_to_element(el)
            time.sleep(4)
            action.click(el)
            action.perform()
        except:
            print("err")

In [129]:
def clickShow2(driver):
    driver.find_element_by_class_name("_3j0u").click()
    elements = driver.find_elements_by_class_name("_4ssp")
    for i in range(5):
        action = action_chains.ActionChains(driver)
        try:
            elements = driver.find_elements_by_class_name("_4ssp")
            print(elements[i])
            action.move_to_element(elements[0])
            time.sleep(4)
            action.click(elements[0])
            action.perform()
            time.sleep(4)
        except:
            print("err ", i)

In [143]:
url_page = "https://www.facebook.com/airbus/"
url_page2 = "https://www.facebook.com/pg/airbus/posts/"
lien_driver = '/usr/local/bin/phantomjs'
#lien_driver = "/home/sid2018-4/Téléchargements/chromedriver"
classe = "_1xnd"
soup, driver = dynamicSoup(url_page2, lien_driver, 4)
clickShow(driver)

In [144]:
#commentaires = driver.find_elements_by_xpath("//div [@data-testid='UFI2Comment/body']")
commentaires = driver.find_elements_by_class_name('_72vr')
#commentaires = soup.find_all('_6qw3')
print('nombre de commentaires trouvés :', str(len(commentaires)))

for com in commentaires:
    print(com.text)

nombre de commentaires trouvés : 0


In [125]:
driver.find_elements_by_class_name("_4ssp")[1]

IndexError: list index out of range

In [132]:
soup

<!DOCTYPE html>
<html class="" id="facebook" lang="fr"><head><meta charset="utf-8"/><meta content="default" id="meta_referrer" name="referrer"/><script>window._cstart=+new Date();</script><script>function envFlush(a){function b(b){for(var c in a)b[c]=a[c]}window.requireLazy?window.requireLazy(["Env"],b):(window.Env=window.Env||{},b(window.Env))}envFlush({"defer_cookies":true,"ajaxpipe_token":"AXhE1ieQzRN_YtoC","timeslice_heartbeat_config":{"pollIntervalMs":33,"idleGapThresholdMs":60,"ignoredTimesliceNames":{"requestAnimationFrame":true,"Event listenHandler mousemove":true,"Event listenHandler mouseover":true,"Event listenHandler mouseout":true,"Event listenHandler scroll":true},"isHeartbeatEnabled":true,"isArtilleryOn":false},"shouldLogCounters":true,"timeslice_categories":{"react_render":true,"reflow":true},"sample_continuation_stacktraces":true,"dom_mutation_flag":true,"khsh":"0`sj`e`rm`s-0fdu^gshdoer-0gc^eurf-3gc^eurf;1;enbtldou;fduDmdldourCxO`ld-2YLMIuuqSdptdru;qsnunuxqd;rdoe-0unjd

In [93]:
commentaire = soup.find_elements_by_xpath("//div [@data-testid='UFI2Comment/body']")

TypeError: 'NoneType' object is not callable

In [ ]:
print('nombre de commentaires trouvés :', str(len(commentaire)))

for com in commentaire:
    print(com.text)